In [1]:
import pandas as pd
import numpy as np

# col_range = [i for i in range(0,28)]
# By default the "date" column was in string format, we need to convert it into date-time format 
# parse_dates =["date"], converts the "date" column to date-time format 
  
# Resampling works with time-series data only 
# so convert "date" column to index
df = pd.read_csv("/content/sample_data/covid19_cases_switzerland_openzh-phase2.csv", usecols= np.r_[0:28], parse_dates =["Date"]) # First 27 columns

df.tail()

,Date,AG,AI,AR,BE,BL,BS,FR,GE,GL,GR,JU,LU,NE,NW,OW,SG,SH,SO,SZ,TG,TI,UR,VD,VS,ZG,ZH,CH
143,2020-10-21,3250.0,154.0,304.0,5043.0,899.0,751.0,3583.0,7492.0,181.0,775.0,833.0,1754.0,1821.0,179.0,184.0,3509.0,259.0,955.0,1538.0,1185.0,1594.0,169.0,11065.0,4835.0,873.0,10441.0,63626.0
144,2020-10-22,3516.0,172.0,330.0,5595.0,965.0,826.0,3892.0,8184.0,187.0,867.0,878.0,1935.0,1835.0,195.0,193.0,3629.0,290.0,1056.0,1654.0,1237.0,1717.0,NaN,11857.0,4868.0,947.0,11222.0,68216.0
145,2020-10-23,NaN,187.0,382.0,6397.0,1063.0,900.0,4390.0,NaN,201.0,952.0,962.0,2119.0,NaN,211.0,200.0,NaN,317.0,1156.0,1759.0,1237.0,2012.0,189.0,12601.0,NaN,1010.0,11877.0,72154.0
146,2020-10-24,NaN,197.0,NaN,7112.0,1182.0,1003.0,4885.0,NaN,NaN,1042.0,NaN,NaN,NaN,233.0,NaN,NaN,NaN,NaN,1858.0,NaN,2269.0,NaN,NaN,NaN,NaN,NaN,74064.0
147,2020-10-25,NaN,NaN,NaN,7646.0,NaN,1084.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2525.0,NaN,NaN,NaN,NaN,NaN,74935.0


In [2]:
df.shape # setting date as index causes decrement in column clount 28 to 27

(148, 28)

In [3]:
df.info() # get metadata of the dataframe

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148 entries, 0 to 147
Data columns (total 28 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    148 non-null    datetime64[ns]
 1   AG      145 non-null    float64       
 2   AI      82 non-null     float64       
 3   AR      69 non-null     float64       
 4   BE      134 non-null    float64       
 5   BL      147 non-null    float64       
 6   BS      148 non-null    float64       
 7   FR      147 non-null    float64       
 8   GE      145 non-null    float64       
 9   GL      104 non-null    float64       
 10  GR      147 non-null    float64       
 11  JU      146 non-null    float64       
 12  LU      115 non-null    float64       
 13  NE      145 non-null    float64       
 14  NW      147 non-null    float64       
 15  OW      101 non-null    float64       
 16  SG      145 non-null    float64       
 17  SH      104 non-null    float64       
 18  SO      99

In [4]:
df.set_index(['Date'],inplace=True) # set index
df[0:1] = 0 #initialize first row with zero

In [5]:
df.shape

(148, 27)

In [6]:
df.drop(columns=['CH'],inplace=True, axis=1) # drop last column
df.shape

(148, 26)

In [7]:
df.head()

,AG,AI,AR,BE,BL,BS,FR,GE,GL,GR,JU,LU,NE,NW,OW,SG,SH,SO,SZ,TG,TI,UR,VD,VS,ZG,ZH
Date,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-05-31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-06-01,0.0,NaN,NaN,0.0,1.0,0.0,0.0,1.0,NaN,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2020-06-02,0.0,NaN,0.0,1.0,1.0,0.0,2.0,4.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,4.0,2.0,0.0,2.0
2020-06-03,1.0,0.0,0.0,2.0,2.0,1.0,3.0,6.0,0.0,1.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,NaN,6.0,2.0,0.0,5.0
2020-06-04,3.0,NaN,0.0,4.0,2.0,1.0,5.0,8.0,0.0,1.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,1.0,NaN,9.0,4.0,0.0,13.0


In [8]:
np.isnan(df).sum() # count of nan values
df.shape

(148, 26)

In [9]:
def fill_missing(df): # fill missing values with previous day case count
    for row in range(df.shape[0]):
        for col in range(df.shape[1]):
            if np.isnan(df[row, col]):
                df[row, col] = df[row-1, col]        

fill_missing(df.values)

np.isnan(df).sum() # now count should be zero


AG    0
AI    0
AR    0
BE    0
BL    0
BS    0
FR    0
GE    0
GL    0
GR    0
JU    0
LU    0
NE    0
NW    0
OW    0
SG    0
SH    0
SO    0
SZ    0
TG    0
TI    0
UR    0
VD    0
VS    0
ZG    0
ZH    0
dtype: int64

In [10]:
df.to_csv('/content/sample_data/exported.csv', index = True) # save raw data in file

In [11]:
daily_changes_df = pd.DataFrame() # Calculate daily case changes e.g. new case on Tuesday = case on Tuesday - case on Monday

def calculate_daily_changes(data_toBe_treated):
  for row in range(data_toBe_treated.shape[0]):
    for col in range(data_toBe_treated.shape[1]):
      daily_changes_df.loc[row, col] = data_toBe_treated[row, col] - data_toBe_treated[row-1, col]


calculate_daily_changes(df.values)

daily_changes_df.columns = ['AG', 'AI', 'AR', 'BE', 'BL', 'BS', 'FR', 'GE', 'GL', 'GR', 'JU', 'LU',
        'NE', 'NW', 'OW', 'SG', 'SH', 'SO', 'SZ', 'TG', 'TI', 'UR', 'VD', 'VS',
        'ZG', 'ZH']

daily_changes_df.index=df.index

daily_changes_df.fillna(0) # Fill null with zeros as well
daily_changes_df[0:1] = 0.0 # Fill null with zeros as well for first row only

In [12]:
daily_changes_df.to_csv('/content/sample_data/changes_daily.csv', index = True) # save case count change in file

In [13]:
dnc_avg = daily_changes_df.rolling(3).mean() # Smooth daily new case by the average value in a rolling window
dnc_avg.fillna(0)

# nan to zero
dnc_avg = dnc_avg.replace(np.nan, 0)

#rouding and converting to int
dnc_avg = dnc_avg.round().astype(int)

dnc_avg.to_csv('/content/sample_data/average_cases.csv', index = True) # save smooth data in file

dnc_avg

,AG,AI,AR,BE,BL,BS,FR,GE,GL,GR,JU,LU,NE,NW,OW,SG,SH,SO,SZ,TG,TI,UR,VD,VS,ZG,ZH
Date,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-05-31,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2020-06-01,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2020-06-02,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,1
2020-06-03,0,0,0,1,1,0,1,2,0,0,0,0,1,0,0,0,0,2,0,0,0,0,2,1,0,2
2020-06-04,1,0,0,1,0,0,2,2,0,0,0,0,2,0,0,0,0,2,0,0,0,0,3,1,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-10-21,205,21,33,321,64,31,243,462,20,39,64,199,157,12,15,347,20,92,143,103,139,21,683,478,77,506
2020-10-22,264,16,29,437,72,46,277,573,12,63,67,170,97,14,8,258,19,73,122,108,172,10,755,302,62,698
2020-10-23,179,19,35,622,79,64,365,354,13,79,70,177,43,16,10,144,23,88,127,56,224,15,773,132,66,808


In [14]:
cantons = df.columns # display final canton and respective dates
date = df.index.strftime('%m/%Y')

cantons, date

(Index(['AG', 'AI', 'AR', 'BE', 'BL', 'BS', 'FR', 'GE', 'GL', 'GR', 'JU', 'LU',
        'NE', 'NW', 'OW', 'SG', 'SH', 'SO', 'SZ', 'TG', 'TI', 'UR', 'VD', 'VS',
        'ZG', 'ZH'],
       dtype='object'),
 Index(['05/2020', '06/2020', '06/2020', '06/2020', '06/2020', '06/2020',
        '06/2020', '06/2020', '06/2020', '06/2020',
        ...
        '10/2020', '10/2020', '10/2020', '10/2020', '10/2020', '10/2020',
        '10/2020', '10/2020', '10/2020', '10/2020'],
       dtype='object', name='Date', length=148))

In [15]:
from bokeh.plotting import figure, show, output_notebook, save # plot covid cases change
from bokeh.models import ColumnDataSource, HoverTool

output_notebook()

source= ColumnDataSource(dnc_avg)

p = figure(plot_width=1000, plot_height=800, x_axis_type="datetime", toolbar_location=None)

# custom color pallete, one color for each canton
color_palette = ['#31446B','#084594','#ff7f0e','#aec7e8','#2ca02c','#d62728','#9467bd','#8c564b','#e377c2','#7f7f7f','#bcbd22','#17becf','#9c9ede','#637939','#cedb9c','#8c6d31','#bd9e39','#e7ba52','#843c39','#de9ed6','#d62728','#e377c2','#17becf','#7f7f7f','#bdbdbd']
# plot line for each canton
for canton,color_pl in zip(cantons,color_palette): 
	p.line(x='Date', y=canton, line_width=1, source=source,color=color_pl, legend_label=canton,name=canton)
 
p.title.text = 'Daily New Cases in Switzerland'
p.legend.location = "top_left"
p.legend.click_policy="hide"
hover = p.select(dict(type=HoverTool))
hover = HoverTool(tooltips=[('Date', '@Date{%F}'),('Cantons','$name'),("Cases", "$y{0}")], formatters={'@Date': 'datetime'})
p.add_tools(hover)
mode='vline'

show(p)